In [ ]:
% pip install psycopg2 # postgreSQL
% pip install pymongo # mongoDB
% pip install redis #redis
% pip install azure-search-documents==11.4.0a20230509004 # ACS dev feed
% pip install openai
% pip install python-dotenv
% pip install langchain

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### 1. PostgreSQL

#### Pgvector extension on Azure Cosmos DB for PostgreSQL

- Database client for accessing Postgre SQL (GUI)
- https://www.dbvis.com/

1. How to configure vector extension

    ```postgresql
    SELECT CREATE_EXTENSION('vector');
    ```

    To disable an extension use drop_extension()

1. pgvector introduces 3 new operators that can be used to calculate similarity:

    | Operator   | Description           |
    |:----------:|:---------------------:|
    |<->	     |Euclidean distance     |
    |<#>	     |negative inner product |
    |<=>	     |cosine distance        |

#### Using native wiring API for postgreSQL

In [ ]:
import psycopg2

# Connect to your PostgreSQL database
conn = psycopg2.connect(
    host="your_host",
    database="your_database",
    user="your_user",
    password="your_password"
)

# Create a cursor object
cur = conn.cursor()

# Execute the first query to create the table
cur.execute("""
    CREATE TABLE tblvector(
        id bigserial PRIMARY KEY,
        embedding vector(3)
    );
""")

# Execute the second query to insert data into the table
cur.execute("""
    INSERT INTO tblvector (id, embedding) VALUES (1, '[1,2,3]'), (2, '[4,5,6]'), (3, '[5,4,6]'), (4, '[3,5,7]'), (5, '[7,8,9]');
""")

# Execute the third query to insert or update data in the table
cur.execute("""
    INSERT INTO tblvector (id, embedding) VALUES (1, '[1,2,3]'), (2, '[4,5,6]')
    ON CONFLICT (id) DO UPDATE SET embedding = EXCLUDED.embedding;
""")

# Execute the fourth query to delete data from the table
# cur.execute("""
#    DELETE FROM tblvector WHERE id = 1;
# """)

# Execute the fifth query to select data from the table
cur.execute("""
    SELECT * FROM tblvector 
    ORDER BY embedding <-> '[3,1,2]' 
    LIMIT 5;
""")

# Fetch and print the results of the SELECT query
results = cur.fetchall()
for row in results:
    print(row)

# Close the cursor and connection
cur.close()
conn.close()


### 2. Azure Cosmos DB for MongoDB vCore


#### Using native wiring API for MongoDB

In [ ]:
import pymongo

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["test"]
exampleCollection = db["exampleCollection"]

# Create indexes
db.command({
  'createIndexes': 'exampleCollection',
  'indexes': [
    {
      'name': 'vectorSearchIndex',
      'key': {
        "vectorContent": "cosmosSearch"
      },
      'cosmosSearchOptions': {
        'kind': 'vector-ivf',
        'numLists': 100,
        'similarity': 'COS',
        'dimensions': 3
      }
    }
  ]
})

# Insert data
exampleCollection.insert_many([
  {'name': "Eugenia Lopez", 'bio': "Eugenia is the CEO of AdvenureWorks.", 'vectorContent': [0.51, 0.12, 0.23]},
  {'name': "Cameron Baker", 'bio': "Cameron Baker CFO of AdvenureWorks.", 'vectorContent': [0.55, 0.89, 0.44]},
  {'name': "Jessie Irwin", 'bio': "Jessie Irwin is the former CEO of AdventureWorks and now the director of the Our Planet initiative.", 'vectorContent': [0.13, 0.92, 0.85]},
  {'name': "Rory Nguyen", 'bio': "Rory Nguyen is the founder of AdventureWorks and the president of the Our Planet initiative.", 'vectorContent': [0.91, 0.76, 0.83]},
])

# Query data
queryVector = [0.52, 0.28, 0.12]
exampleCollection.aggregate([
  {
    '$search': {
      "cosmosSearch": {
        "vector": queryVector,
        "path": "vectorContent",
        "k": 2
      },
    "returnStoredSource": True
    }
  }
])

# Get metadata
exampleCollection.index_information()


### 3. Azure Cognitive Search

#### Vector search (private preview) - Azure Cognitive Search

In [ ]:
% pip install azure-search-documents==11.4.0a20230509004
% pip install openai
% pip install python-dotenv

In [ ]:
# Import required libraries  
import os  
import json  
import openai  
from dotenv import load_dotenv  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    VectorSearchAlgorithmConfiguration,  
)  
  
# Configure environment variables  
load_dotenv()  
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")  
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")  
key = os.getenv("AZURE_SEARCH_ADMIN_KEY")  
openai.api_type = "azure"  
openai.api_key = os.getenv("OPENAI_API_KEY")  
openai.api_base = os.getenv("OPENAI_ENDPOINT")  
openai.api_version = os.getenv("OPENAI_API_VERSION")  
credential = AzureKeyCredential(key)

In [ ]:
# Generate Document Embeddings using OpenAI Ada 002

# Read the text-sample.json
with open('../data/text-sample.json', 'r', encoding='utf-8') as file:
    input_data = json.load(file)

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields, also used for query embeddings
def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text, engine="text-embedding-ada-002")
    embeddings = response['data'][0]['embedding']
    return embeddings


# Generate embeddings for title and content fields
for item in input_data:
    title = item['title']
    content = item['content']
    title_embeddings = generate_embeddings(title)
    content_embeddings = generate_embeddings(content)
    item['titleVector'] = title_embeddings
    item['contentVector'] = content_embeddings
    item['@search.action'] = 'upload'

# Output embeddings to docVectors.json file
with open("../output/docVectors.json", "w") as f:
    json.dump(input_data, f)

In [ ]:
# Create a search index
index_client = SearchIndexClient(
    endpoint=service_endpoint, credential=credential)
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="title", type=SearchFieldDataType.String,
                    searchable=True, retrievable=True),
    SearchableField(name="content", type=SearchFieldDataType.String,
                    searchable=True, retrievable=True),
    SearchableField(name="category", type=SearchFieldDataType.String,
                    filterable=True, searchable=True, retrievable=True),
    SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, dimensions=1536, vector_search_configuration="my-vector-config"),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, dimensions=1536, vector_search_configuration="my-vector-config"),
]

vector_search = VectorSearch(
    algorithm_configurations=[
        VectorSearchAlgorithmConfiguration(
            name="my-vector-config",
            kind="hnsw",
            hnsw_parameters={
                "m": 4,
                "efConstruction": 400,
                "efSearch": 1000,
                "metric": "cosine"
            }
        )
    ]
)

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=PrioritizedFields(
        title_field=SemanticField(field_name="title"),
        prioritized_keywords_fields=[SemanticField(field_name="category")],
        prioritized_content_fields=[SemanticField(field_name="content")]
    )
)

# Create the semantic settings with the configuration
semantic_settings = SemanticSettings(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_settings=semantic_settings)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')


In [ ]:
# Upload some documents to the index
with open('../output/docVectors.json', 'r') as file:  
    documents = json.load(file)  
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)
result = search_client.upload_documents(documents)  
print(f"Uploaded {len(documents)} documents") 

In [ ]:
# Pure Vector Search
query = "tools for software development"  
  
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))  
  
results = search_client.search(  
    search_text="",  
    vector=Vector(value=generate_embeddings(query), k=3, fields="contentVector"),  
    select=["title", "content", "category"] 
)  
  
for result in results:  
    print(f"Title: {result['title']}")  
    print(f"Content: {result['content']}")  
    print(f"Category: {result['category']}\n")  


### 4. Azure Cache for Redis

#### Azure Cache for Redis Enterprise

https://redis.readthedocs.io/en/latest/examples/search_vector_similarity_examples.html

In [ ]:
import redis
from redis.commands.search.field import TagField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query

r = redis.Redis(host="localhost", port=6379)

INDEX_NAME = "index"                              # Vector Index Name
DOC_PREFIX = "doc:"                               # RediSearch Key Prefix for the Index

def create_index(vector_dimensions: int):
    try:
        # check to see if index exists
        r.ft(INDEX_NAME).info()
        print("Index already exists!")
    except:
        # schema
        schema = (
            TagField("tag"),                       # Tag Field Name
            VectorField("vector",                  # Vector Field Name
                "FLAT", {                          # Vector Index Type: FLAT or HNSW
                    "TYPE": "FLOAT32",             # FLOAT32 or FLOAT64
                    "DIM": vector_dimensions,      # Number of Vector Dimensions
                    "DISTANCE_METRIC": "COSINE",   # Vector Search Distance Metric
                }
            ),
        )

        # index Definition
        definition = IndexDefinition(prefix=[DOC_PREFIX], index_type=IndexType.HASH)

        # create Index
        r.ft(INDEX_NAME).create_index(fields=schema, definition=definition)

In [ ]:
# define vector dimensions
VECTOR_DIMENSIONS = 1536

# create the index
create_index(vector_dimensions=VECTOR_DIMENSIONS)

In [ ]:
%pip install openai

In [ ]:
import openai

# set your OpenAI API key - get one at https://platform.openai.com
openai.api_key = "YOUR OPENAI API KEY"
# Create Embeddings with OpenAI text-embedding-ada-002
# https://openai.com/blog/new-and-improved-embedding-model
response = openai.Embedding.create(input=texts, engine="text-embedding-ada-002")
embeddings = np.array([r["embedding"] for r in response["data"]], dtype=np.float32)

# Write to Redis
pipe = r.pipeline()
for i, embedding in enumerate(embeddings):
    pipe.hset(f"doc:{i}", mapping = {
        "vector": embedding.tobytes(),
        "content": texts[i],
        "tag": "openai"
    })
res = pipe.execute()

In [ ]:
text = "animals"

# create query embedding
response = openai.Embedding.create(input=[text], engine="text-embedding-ada-002")
query_embedding = np.array([r["embedding"] for r in response["data"]], dtype=np.float32)[0]

In [ ]:
# query for similar documents that have the openai tag
query = (
    Query("(@tag:{ openai })=>[KNN 2 @vector $vec as score]")
     .sort_by("score")
     .return_fields("content", "tag", "score")
     .paging(0, 2)
     .dialect(2)
)

query_params = {"vec": query_embedding.tobytes()}
r.ft(INDEX_NAME).search(query, query_params).docs

# the two pieces of content related to animals are returned